# 🤖 Entrenamiento del Modelo Predictivo (PyCaret)

## 🎯 Objetivo
Este notebook orquesta el pipeline de entrenamiento de Machine Learning utilizando **PyCaret**.
El objetivo es encontrar y optimizar el mejor algoritmo capaz de predecir la probabilidad de **Enfermedad Cardíaca** basándose en biomarcadores clínicos.

## ⚙️ Estrategia de Modelado
1. **Preprocesamiento Robusto**: Normalización y manejo de outliers.
2. **Balanceo de Clases**: Uso de técnicas (SMOTE) para mitigar el desbalance entre pacientes sanos y enfermos.
3. **Optimización de Recall**: Priorizamos la **Sensibilidad (Recall)** sobre la Precisión.
   - *Contexto Médico*: Es peor no detectar a un enfermo (Falso Negativo) que alarmar a un sano (Falso Positivo).
4. **Selección de Modelos**: Comparación automática de +15 algoritmos.

## 📂 Entradas y Salidas
- **Input**: `data/02_intermediate/process_data.parquet` (Datos limpios).
- **Output**: `models/best_pipeline.pkl` (Modelo serializado listo para producción).

## 1. Configuración del Entorno

Definimos parámetros globales.
- **SAMPLE_FRAC**: Porcentaje de datos a usar. Para pruebas rápidas usamos `0.5`, para el modelo final debe ser `1.0`.
- **Rutas**: Ubicación de datos y donde se guardarán los artefactos.

### 🔹 Paso 1: Configuración del Entorno y Constantes
Inicializamos el entorno de trabajo importando **PyCaret** y definiendo constantes críticas:
- `SAMPLE_FRAC`: Controla el muestreo de datos. Usamos 0.5 (50%) para iteraciones rápidas de desarrollo, pero se debe cambiar a 1.0 para el entrenamiento final.
- `DATA_PATH` y `MODEL_DIR`: Definen las rutas de entrada de datos y salida del modelo, asegurando una estructura de proyecto ordenada.

In [ ]:
import pandas as pd
from pycaret.classification import *
import os
import json

# ==========================================
# CONFIGURATION
# ==========================================
SAMPLE_FRAC = 0.5  # Set to 1.0 for full training
DATA_PATH = "../data/02_intermediate/process_data.parquet"
MODEL_DIR = "../models"
MODEL_NAME = "best_pipeline"
CONFIG_PATH = "../models/model_config.json"

print(f"Running Training with SAMPLE_FRAC = {SAMPLE_FRAC}")

## 2. Carga y Filtrado de Datos

Cargamos el dataset y aplicamos el esquema definido en `model_config.json`.
Es vital entrenar **solo** con las columnas que estarán disponibles en la aplicación final (Features + Target), descartando metadatos o IDs que causarían *data leakage*.

### 🔹 Paso 2: Carga y Selección de Features (Data Loading)
Cargamos el dataset procesado y aplicamos un filtro estricto de columnas basado en `model_config.json`.
**Importante**:
- Solo cargamos las columnas definidas como `features` y el `target`.
- Esto actúa como una barrera de seguridad contra el *data leakage*, asegurando que el modelo no vea variables que no estarán disponibles en producción (como IDs de pacientes o fechas de procesamiento).

In [ ]:
# ==========================================
# 1. LOAD DATA
# ==========================================
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"Data file not found at {DATA_PATH}")

df = pd.read_parquet(DATA_PATH)
print(f"Original Data Shape: {df.shape}")

# Load Schema Config
with open(CONFIG_PATH, 'r') as f:
    config = json.load(f)

features = config['features']
target = config['target']
numeric_features = config['numeric_features']
categorical_features = config['categorical_features']

# Filter only relevant columns
df = df[features + [target]]

if SAMPLE_FRAC < 1.0:
    df = df.sample(frac=SAMPLE_FRAC, random_state=42)
    print(f"Sampled Data Shape: {df.shape}")
else:
    print("Using Full Dataset")


## 3. Configuración del Experimento (Setup)

La función `setup()` inicializa el entorno de PyCaret y crea el pipeline de transformación.
- **normalize=True**: Escala las variables para que tengan rangos comparables. Usamos `RobustScaler` para ser resilientes a outliers.
- **remove_outliers=True**: Elimina anomalías estadísticas que podrían sesgar el modelo.
- **fix_imbalance=True**: Aplica SMOTE para generar muestras sintéticas de la clase minoritaria (Enfermos), mejorando el aprendizaje.

### 🔹 Paso 3: Inicialización del Experimento (PyCaret Setup)
Configuramos el pipeline de preprocesamiento automático con `setup()`. Aquí definimos la "magia" de PyCaret:
- **Normalización**: Aplicamos `RobustScaler` (`normalize_method='robust'`) para escalar los datos manejando bien los outliers típicos de datos clínicos.
- **Balanceo de Clases**: Activamos `fix_imbalance=True` (SMOTE) para generar datos sintéticos de la clase minoritaria (pacientes enfermos), evitando que el modelo se sesgue hacia la clase mayoritaria (sanos).
- **Tipos de Datos**: Definimos explícitamente cuáles son numéricas y cuáles categóricas.

In [ ]:
# ==========================================
# 2. SETUP PYCARET
# ==========================================
# normalize=True (RobustScaler)
# remove_outliers=True
# fix_imbalance=True

exp = setup(
    data=df,
    target=target,
    numeric_features=numeric_features,
    categorical_features=categorical_features,
    normalize=True,
    normalize_method='robust',
    remove_outliers=True,
    fix_imbalance=True,
    session_id=123,
    verbose=True
)

## 4. Comparación y Selección de Modelos

Entrenamos múltiples algoritmos (Logistic Regression, XGBoost, Random Forest, etc.) con validación cruzada (Cross-Validation).
**Métrica Clave: Recall**. Buscamos maximizar la capacidad del modelo para detectar casos positivos reales.

### 🔹 Paso 4: Entrenamiento y Comparación de Modelos
Ejecutamos `compare_models()` para entrenar y evaluar múltiples algoritmos (Random Forest, XGBoost, LightGBM, etc.) simultáneamente.
**Criterio de Selección**: Ordenamos por `Recall` (`sort='Recall'`).
**Justificación Clínica**: En el diagnóstico de enfermedades, priorizamos minimizar los Falsos Negativos. Preferimos que el modelo tenga "falsas alarmas" (baja precisión) a que deje pasar un caso de enfermedad cardíaca real (alto recall).

In [ ]:
# ==========================================
# 3. COMPARE & TRAIN
# ==========================================
# Optimizing for Recall (Sensitivity) as per medical requirements
best_model = compare_models(sort='Recall')
print("Best Model Found:")
print(best_model)

## 5. Finalización y Persistencia

Una vez seleccionado el mejor modelo:
1. **Finalize**: Se re-entrena el modelo utilizando el 100% de los datos (incluyendo el set de prueba reservado anteriormente).
2. **Save**: Se guarda el pipeline completo (preprocesamiento + modelo) en un archivo `.pkl` para su despliegue en la API/Streamlit.

### 🔹 Paso 5: Finalización y Serialización del Modelo
Una vez seleccionado el mejor algoritmo:
1.  **Finalize**: Re-entrenamos el modelo utilizando **todos** los datos disponibles (incluyendo el set de validación que PyCaret retuvo internamente).
2.  **Save**: Guardamos el pipeline completo como un archivo `.pkl` en el directorio `models/`. Este archivo contiene tanto el modelo predictivo como las transformaciones de datos (escalado, imputación), listo para ser consumido por la API.

In [ ]:
# ==========================================
# 4. FINALIZE & SAVE
# ==========================================
final_model = finalize_model(best_model)
os.makedirs(MODEL_DIR, exist_ok=True)
save_path = os.path.join(MODEL_DIR, MODEL_NAME)
save_model(final_model, save_path)
print(f"Model saved successfully to {save_path}.pkl")